## Отображение всех заведений общепита на карте Москвы

Способ из статьи + датафрейм от Яндекс Практикума: https://medium.com/nuances-of-programming/python-selenium-%D0%BA%D0%B0%D0%BA-%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D1%82%D1%8C-%D0%BA%D0%BE%D0%BE%D1%80%D0%B4%D0%B8%D0%BD%D0%B0%D1%82%D1%8B-%D0%BF%D0%BE-%D0%B0%D0%B4%D1%80%D0%B5%D1%81%D0%B0%D0%BC-ea7a78ffdc0d

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Файл с заведениями:

In [8]:
df = pd.read_csv('rest_data.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB


In [10]:
# удалила дубликаты
df.drop_duplicates(['object_name','chain', 'object_type', 'address', 'number'], inplace=True)

In [11]:
# перевела все названия и адреса в нижний регистр
df['object_name'] = df['object_name'].str.lower()
df['address'] = df['address'].str.lower()

### Устанавливаем Selenium 

In [12]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver

### Устанавливаем Folium

In [14]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [15]:
import folium

In [20]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [21]:
from tqdm import tqdm_notebook as tqdmn

Создаем новый столбец с url. В нем комбинация www.google.com/maps/search/ и каждого адреса (address) из нашего датафрейма.

In [16]:
df['url'] = ['https://www.google.com/maps/search/' + i for i in df['address']]

In [17]:
df.head()

,id,object_name,chain,object_type,address,number,url
0,151635,сметана,нет,кафе,"город москва, улица егора абакумова, дом 9",48,https://www.google.com/maps/search/город москв...
1,77874,родник,нет,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,https://www.google.com/maps/search/город москв...
2,24309,кафе «академия»,нет,кафе,"город москва, абельмановская улица, дом 6",95,https://www.google.com/maps/search/город москв...
3,21894,пиццетория,да,кафе,"город москва, абрамцевская улица, дом 1",40,https://www.google.com/maps/search/город москв...
4,119365,кафе «вишневая метель»,нет,кафе,"город москва, абрамцевская улица, дом 9, корпус 1",50,https://www.google.com/maps/search/город москв...


Извлекаем URL с координатами из Google Maps (скрипт выполнялся около 3 часов)

In [22]:
Url_With_Coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)


driver = webdriver.Chrome(executable_path="C:\chromedriver.exe", options=option)

for url in tqdmn(df.url, leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
    
driver.close()

In [25]:
import csv

In [26]:
with open('Url_With_Coordinates.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates)

In [27]:
with open('Url_With_Coordinates.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        Url_With_Coordinates = i
        break

In [28]:
Url_With_Coordinates[:5]

['https://maps.google.com/maps/api/staticmap?center=55.8792377%2C37.7141957&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=B1wuaf2Nio7HcS31WtXlas_HE34',
 'https://maps.google.com/maps/api/staticmap?center=55.7376317%2C37.6748651&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=_wvOk3x6bE0vKT41LqxACJ0yBGk',
 'https://maps.google.com/maps/api/staticmap?center=55.7355%2C37.669608&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=JyHmjuYdT5lxJlWgUIKN9YbPgIs',
 'https://maps.google.com/maps/api/staticmap?center=55.8917906%2C37.5734207&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=KtLhsEmzlzG0ic2hMtHZSAyeURA',
 'https://maps.google.com/maps/api/staticmap?center=55.9049383%2C37.572052&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=gfqT4Cpscvfd0Pfp0KjyvwYgGNs']

Создаем столбец с url с координатами

In [29]:
df['Url_With_Coordinates'] = Url_With_Coordinates

Вынимаем широту и долготу

In [30]:
df['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in df['Url_With_Coordinates'] ]
df['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in df['Url_With_Coordinates'] ]

IndexError: list index out of range

Эта ошибка говорит о том, что метод split() не сработал так, как ожидалось, на некоторых URL в колонке Url_With_Coordinates. Возможно, в некоторых URL не было ключевых слов, которые мы задавали для метода split().

In [31]:
df[~df.Url_With_Coordinates.str.contains('&zoom=')]

,id,object_name,chain,object_type,address,number,url,Url_With_Coordinates
1888,150706,кафе щапово,нет,кафе,"город москва, поселение щаповское, посёлок щап...",20,https://www.google.com/maps/search/город москв...,//www.gstatic.com/images/branding/product/1x/m...
8842,19800,набатчиков а.п.,нет,буфет,"проектируемый проезд n 5231, дом 8, строение 3",20,https://www.google.com/maps/search/проектируем...,//www.gstatic.com/images/branding/product/1x/m...
9773,169618,"выпечка, пиво",нет,предприятие быстрого обслуживания,"город москва, поселение ""мосрентген"", посёлок ...",3,https://www.google.com/maps/search/город москв...,//www.gstatic.com/images/branding/product/1x/m...
10176,172496,суши wok,да,кафе,"город москва, бауманская улица, дом 33/2, стро...",10,https://www.google.com/maps/search/город москв...,//www.gstatic.com/images/branding/product/1x/m...
15207,223318,баскин роббинс,да,закусочная,"город москва, шоссе энтузиастов, дом 12, корпус 2",0,https://www.google.com/maps/search/город москв...,//www.gstatic.com/images/branding/product/1x/m...


Убираем эти значения (начинающиеся с gstatic)

In [32]:
df = df[df.Url_With_Coordinates.str.contains('&zoom=')].copy()

In [33]:
df['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in df['Url_With_Coordinates'] ]
df['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in df['Url_With_Coordinates'] ]

У каждого заведения есть свои широта и долгота

In [34]:
df.head()

,id,object_name,chain,object_type,address,number,url,Url_With_Coordinates,lat,long
0,151635,сметана,нет,кафе,"город москва, улица егора абакумова, дом 9",48,https://www.google.com/maps/search/город москв...,https://maps.google.com/maps/api/staticmap?cen...,55.8792377,37.7141957
1,77874,родник,нет,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,https://www.google.com/maps/search/город москв...,https://maps.google.com/maps/api/staticmap?cen...,55.7376317,37.6748651
2,24309,кафе «академия»,нет,кафе,"город москва, абельмановская улица, дом 6",95,https://www.google.com/maps/search/город москв...,https://maps.google.com/maps/api/staticmap?cen...,55.7355,37.669608
3,21894,пиццетория,да,кафе,"город москва, абрамцевская улица, дом 1",40,https://www.google.com/maps/search/город москв...,https://maps.google.com/maps/api/staticmap?cen...,55.8917906,37.5734207
4,119365,кафе «вишневая метель»,нет,кафе,"город москва, абрамцевская улица, дом 9, корпус 1",50,https://www.google.com/maps/search/город москв...,https://maps.google.com/maps/api/staticmap?cen...,55.9049383,37.572052


### Наносим координаты на карту

In [35]:
from IPython.display import IFrame

In [36]:
df_in_chain  = df[df.chain == 'да']
df_not_in_chain = df[df.chain == 'нет']

In [37]:
df_map = folium.Map(location=[55.764414, 37.647859], tiles='CartoDB positron' )

In [38]:
for lat, long, object_name, address in zip(df_in_chain.lat, df_in_chain.long, df_in_chain.object_name, df_in_chain.address):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://i.imgur.com/CYx04oC.png', icon_size=(8,8) ), popup=object_name+'\n\n'+address ).add_to(df_map)

for lat, long, object_name, address in zip(df_not_in_chain.lat, df_not_in_chain.long, df_not_in_chain.object_name, df_not_in_chain.address):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://imgur.com/Rzs4Zpa.png', icon_size=(8,8) ), popup=object_name+'\n\n'+address ).add_to(df_map)

In [39]:
df_map.save('df_map.html')
IFrame(src='df_map.html', width='100%', height=500)

Фиолетовые - несетевые заведения, малиновые -сетевые.